# Export from Steuerrechner
This script grabs the files and stores them. No analytics or transformation.

In `utils.py` werden die verschiedenen Payload-Möglichkeiten gespeichert (in eigenes file ausgelagert, da diese bei der Analyse wieder gebraucht werden). Wie man zu diesen Werten kommt:  

**MassExportModel ("Steuersubjekt)**  
Exported the ID's and values from the website via chrome console and this script (cleaned by hand):
```javascript
list = document.querySelectorAll('.income-wealth-tax-burden__mass-export-model-select option')
s = ''
list.forEach(e => s += `    ${e.value}: "${e.text}",\n`)
console.log(`{\n${s}\n}`)
```
**Confession**  
By google inspector and by hand (only 4)

In [ ]:
import requests
from pathlib import Path
from fake_useragent import UserAgent
import json
import hashlib

# Edit if necessary
import utils



In [ ]:
# Consts
export_folder = Path('../export/dumps/')


## Prepare Requests

In [ ]:
# Create Fake user Agent
ua = UserAgent()

# Where to grab the data
url = 'https://swisstaxcalculator.estv.admin.ch/delegate/ost-integration/v1/export/income-tax-values/JSON/DE'

headers = {
    'Accept': 'application/json, text/plain, */*',
    'Accept-Encoding': 'gzip, deflate, br',
    'Content-Type': 'application/json;charset=UTF-8',
    'Host': 'swisstaxcalculator.estv.admin.ch',
    'Origin': 'https://swisstaxcalculator.estv.admin.ch'
}

payload = {
    "massExportModelId": 2, # This will be updated
    "belastungsVergleich": 1,
    "taxGroup": 99, # 99: all municipalities
    "confession": 5, # This will be updated
    "incomes": [15000,20000,30000,25000,35000,40000,90000,80000,70000,60000,50000,45000,100000,125000,150000,175000,200000,250000,5000000,1000000,750000,500000,400000,300000,10000000,50000000,100000000],
    "fortunes": [],
    "klasse": 1,
    "years": [2010],#for taxGroup = 99, only one year at once
    "output": 1,
    "considerMaximumExposureLimit": False,
    "investmentIncome": None
}

## The beef: Loop everything
Do it fail safe

In [ ]:
# Loop every massExportModel with every confession

# Loop each "Steuersubjekt"
for model in utils.massExportModel:

    # Loop each "Konfession"
    for confession in utils.confessions:
        
        # Update Payload
        payload['massExportModelId'] = model
        payload['confession'] = confession

        # Update User-Agent in Header for every request (trick the server :) )
        headers['User-Agent'] = ua.random

        # Create Hash from payload
        hash = hashlib.sha1(str(payload).encode('utf-8')).hexdigest()

        filename = export_folder / Path('%s.json' % hash)

        # Check if file already exists. Is so, jump to next one
        if filename.exists():
            continue
        
        # FIRE!!!!!!
        r = requests.post(url, data=json.dumps(payload), headers=headers)

        # convert Requests to json. This will fail if something gone wrong - and thats good
        data = json.loads(r.text)

        # store
        with open(filename, 'w', encoding='utf-8') as f:
            json.dump(data, f, ensure_ascii=False)

print("Finito 🍩")


## Only for confession = 5 (no confession): Without Kirchensteuer

In [ ]:
# Loop each "Steuersubjekt" 

for model in utils.massExportModel:
     
    # Update Payload
    payload['massExportModelId'] = model
    payload['confession'] = 5

    # Update User-Agent in Header for every request (trick the server :) )
    headers['User-Agent'] = ua.random

    # Create Hash from payload
    hash = hashlib.sha1(str(payload).encode('utf-8')).hexdigest()

    filename = export_folder / Path('%s.json' % hash)

    # Check if file already exists. Is so, jump to next one
    if filename.exists():
        continue

    # FIRE!!!!!!
    r = requests.post(url, data=json.dumps(payload), headers=headers)

    # convert Requests to json. This will fail if something gone wrong - and thats good
    data = json.loads(r.text)

    # store
    with open(filename, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False)

print("Finito 🍩")
